In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import Sequential,Model
from keras.layers import concatenate,Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from sklearn.metrics import roc_curve
from keras.utils import np_utils
from tqdm import tqdm
import pandas as pd
import numpy as np

import librosa 
import librosa.display
import pylab
import cv2
import os

Using TensorFlow backend.


In [2]:
names   = ['ID','Fever/MP','ORC','STATUS','DIR','DataSet']
join_by = pd.read_csv('../Coswara-Data-master/coswara/combined_data.csv')

import glob
df_list = []
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/cough-shallow.wav')):
    temp = pd.DataFrame(columns=['id','DIR'])
    temp['id'] = [path.split('/')[-2]]
    temp['DIR'] = [path]
    temp = pd.merge(left=temp,right=join_by,on='id',how='inner')
    temp['fomp']= (temp['fever']| temp['mp']).apply(int)
    temp['oths']= (temp['cld']|temp['asthma']|temp['cold']|temp['st']|temp['pneumonia']).apply(int)
    temp        = temp[['id','covid_status','DIR','fomp','oths']]
    df_list.append(temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
    
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/cough-heavy.wav')):
    temp = pd.DataFrame(columns=['id','DIR'])
    temp['id'] = [path.split('/')[-2]]
    temp['DIR'] = [path]
    temp = pd.merge(left=temp,right=join_by,on='id',how='inner')
    temp['fomp']= (temp['fever']| temp['mp']).apply(int)
    temp['oths']= (temp['cld']|temp['asthma']|temp['cold']|temp['st']|temp['pneumonia']).apply(int)
    temp        = temp[['id','covid_status','DIR','fomp','oths']]
    df_list.append(temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
    
CosData=pd.concat(df_list)
CosData['Cough'] = True
CosData['DataSet'] = 'coswara'
CosData.head()

100%|██████████| 1645/1645 [00:12<00:00, 134.11it/s]


,ID,STATUS,DIR,Fever/MP,ORC,Cough,DataSet
0,5ZDF98UQUxcVuS9VhaWxKlVnXvs1,no_resp_illness_exposed,../Coswara-Data-master/20200413/5ZDF98UQUxcVuS...,0,0,True,coswara
0,FGRDO4IBbAejR0WHD5YbkXTCasg2,healthy,../Coswara-Data-master/20200413/FGRDO4IBbAejR0...,0,0,True,coswara
0,CAf8ILsDpQMeTPPLwi7nFlgRT2t1,healthy,../Coswara-Data-master/20200413/CAf8ILsDpQMeTP...,0,0,True,coswara
0,aGEXEhp3mbUandZBtCuEooDQrK53,healthy,../Coswara-Data-master/20200413/aGEXEhp3mbUand...,0,0,True,coswara
0,uMry1yqi5sNoykw7ySJdITMv2yT2,resp_illness_not_identified,../Coswara-Data-master/20200413/uMry1yqi5sNoyk...,0,0,True,coswara


In [3]:
names   = ['ID','Fever/MP','ORC','STATUS','DIR','DataSet']
join_by = pd.read_csv('../Coswara-Data-master/coswara/combined_data.csv')

import glob
false_df_list = []
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/breathing-deep.wav')):
    false_temp = pd.DataFrame(columns=['id','DIR'])
    false_temp['id'] = [path.split('/')[-2]]
    false_temp['DIR'] = [path]
    false_temp = pd.merge(left=false_temp,right=join_by,on='id',how='inner')
    false_temp['fomp']= (false_temp['fever']| false_temp['mp']).apply(int)
    false_temp['oths']= (false_temp['cld']|false_temp['asthma']|false_temp['cold']|false_temp['st']|false_temp['pneumonia']).apply(int)
    false_temp        = false_temp[['id','covid_status','DIR','fomp','oths']]
    false_df_list.append(false_temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
    
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/breathing-shallow.wav')):
    false_temp = pd.DataFrame(columns=['id','DIR'])
    false_temp['id'] = [path.split('/')[-2]]
    false_temp['DIR'] = [path]
    false_temp = pd.merge(left=false_temp,right=join_by,on='id',how='inner')
    false_temp['fomp']= (false_temp['fever']| false_temp['mp']).apply(int)
    false_temp['oths']= (false_temp['cld']|false_temp['asthma']|false_temp['cold']|false_temp['st']|false_temp['pneumonia']).apply(int)
    false_temp        = false_temp[['id','covid_status','DIR','fomp','oths']]  
    false_df_list.append(false_temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/counting-fast.wav')):
    false_temp = pd.DataFrame(columns=['id','DIR'])
    false_temp['id'] = [path.split('/')[-2]]
    false_temp['DIR'] = [path]
    false_temp = pd.merge(left=false_temp,right=join_by,on='id',how='inner')
    false_temp['fomp']= (false_temp['fever']| false_temp['mp']).apply(int)
    false_temp['oths']= (false_temp['cld']|false_temp['asthma']|false_temp['cold']|false_temp['st']|false_temp['pneumonia']).apply(int)
    false_temp        = false_temp[['id','covid_status','DIR','fomp','oths']]
    false_df_list.append(false_temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/counting-normal.wav')):
    false_temp = pd.DataFrame(columns=['id','DIR'])
    false_temp['id'] = [path.split('/')[-2]]
    false_temp['DIR'] = [path]
    false_temp = pd.merge(left=false_temp,right=join_by,on='id',how='inner')
    false_temp['fomp']= (false_temp['fever']| false_temp['mp']).apply(int)
    false_temp['oths']= (false_temp['cld']|false_temp['asthma']|false_temp['cold']|false_temp['st']|false_temp['pneumonia']).apply(int)
    false_temp        = false_temp[['id','covid_status','DIR','fomp','oths']]
    false_df_list.append(false_temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/vowel-a.wav')):
    false_temp = pd.DataFrame(columns=['id','DIR'])
    false_temp['id'] = [path.split('/')[-2]]
    false_temp['DIR'] = [path]
    false_temp = pd.merge(left=false_temp,right=join_by,on='id',how='inner')
    false_temp['fomp']= (false_temp['fever']| false_temp['mp']).apply(int)
    false_temp['oths']= (false_temp['cld']|false_temp['asthma']|false_temp['cold']|false_temp['st']|false_temp['pneumonia']).apply(int)
    false_temp        = false_temp[['id','covid_status','DIR','fomp','oths']]
    false_df_list.append(false_temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/vowel-e.wav')):
    false_temp = pd.DataFrame(columns=['id','DIR'])
    false_temp['id'] = [path.split('/')[-2]]
    false_temp['DIR'] = [path]
    false_temp = pd.merge(left=false_temp,right=join_by,on='id',how='inner')
    false_temp['fomp']= (false_temp['fever']| false_temp['mp']).apply(int)
    false_temp['oths']= (false_temp['cld']|false_temp['asthma']|false_temp['cold']|false_temp['st']|false_temp['pneumonia']).apply(int)
    false_temp        = false_temp[['id','covid_status','DIR','fomp','oths']]
    false_df_list.append(false_temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))
for path in tqdm(glob.glob('../Coswara-Data-master/*/*/vowel-o.wav')):
    false_temp = pd.DataFrame(columns=['id','DIR'])
    false_temp['id'] = [path.split('/')[-2]]
    false_temp['DIR'] = [path]
    false_temp = pd.merge(left=false_temp,right=join_by,on='id',how='inner')
    false_temp['fomp']= (false_temp['fever']| false_temp['mp']).apply(int)
    false_temp['oths']= (false_temp['cld']|false_temp['asthma']|false_temp['cold']|false_temp['st']|false_temp['pneumonia']).apply(int)
    false_temp        = false_temp[['id','covid_status','DIR','fomp','oths']]
    false_df_list.append(false_temp.rename(columns={'id':'ID','covid_status':'STATUS','DIR':'DIR','fomp':'Fever/MP','oths':'ORC'}))   
    
false_CosData=pd.concat(false_df_list)
false_CosData['Cough'] = False
false_CosData['DataSet'] = 'coswara'
false_CosData.head()

100%|██████████| 1644/1644 [00:12<00:00, 131.90it/s]


,ID,STATUS,DIR,Fever/MP,ORC,Cough,DataSet
0,5ZDF98UQUxcVuS9VhaWxKlVnXvs1,no_resp_illness_exposed,../Coswara-Data-master/20200413/5ZDF98UQUxcVuS...,0,0,False,coswara
0,FGRDO4IBbAejR0WHD5YbkXTCasg2,healthy,../Coswara-Data-master/20200413/FGRDO4IBbAejR0...,0,0,False,coswara
0,CAf8ILsDpQMeTPPLwi7nFlgRT2t1,healthy,../Coswara-Data-master/20200413/CAf8ILsDpQMeTP...,0,0,False,coswara
0,aGEXEhp3mbUandZBtCuEooDQrK53,healthy,../Coswara-Data-master/20200413/aGEXEhp3mbUand...,0,0,False,coswara
0,uMry1yqi5sNoykw7ySJdITMv2yT2,resp_illness_not_identified,../Coswara-Data-master/20200413/uMry1yqi5sNoyk...,0,0,False,coswara


In [4]:
ESC_50  = '../ESC-50-master/audio/'

ESCData  = pd.read_csv("../ESC-50-master/meta/esc50.csv")
ESCData   = ESCData.loc[ESCData['category'] != 'coughing'][['filename','src_file']]
ESCData['DIR'] = ESC_50 + ESCData['filename']
ESCData['ID'] = ESCData['src_file'].astype(str)
ESCData['Cough'] = False
ESCData['DataSet'] = 'ESC-50'
ESCData   = ESCData.sample(frac=1).reset_index(drop=True)
ESCData = ESCData[['ID','DIR','Cough','DataSet']]
ESCData.head()

,ID,DIR,Cough,DataSet
0,203128,../ESC-50-master/audio/5-203128-A-0.wav,False,ESC-50
1,240671,../ESC-50-master/audio/5-240671-A-44.wav,False,ESC-50
2,234247,../ESC-50-master/audio/5-234247-A-37.wav,False,ESC-50
3,183992,../ESC-50-master/audio/4-183992-A-0.wav,False,ESC-50
4,17808,../ESC-50-master/audio/1-17808-A-12.wav,False,ESC-50


In [5]:
coughvid  = './public_dataset/'

TotData  = pd.read_csv(os.path.join(coughvid,'metadata_compiled.csv'),header=0)
TotData   = TotData.loc[TotData['cough_detected'] >= 0.8][['uuid','fever_muscle_pain','respiratory_condition','status']]
TotData['DIR'] = coughvid + TotData['uuid'] + '.wav'
TotData['Cough'] = True
TotData['DataSet'] = 'coughvid'
TotData   = pd.concat([false_CosData,CosData,TotData.rename(columns={'uuid':'ID','status':'STATUS','fever_muscle_pain':'Fever/MP','respiratory_condition':'ORC'})])
TotData   = TotData.sample(frac=1).reset_index(drop=True)
TotData = TotData[['ID','DIR','Cough','DataSet']]
TotData = pd.concat([ESCData,TotData])
TotData.head()

,ID,DIR,Cough,DataSet
0,203128,../ESC-50-master/audio/5-203128-A-0.wav,False,ESC-50
1,240671,../ESC-50-master/audio/5-240671-A-44.wav,False,ESC-50
2,234247,../ESC-50-master/audio/5-234247-A-37.wav,False,ESC-50
3,183992,../ESC-50-master/audio/4-183992-A-0.wav,False,ESC-50
4,17808,../ESC-50-master/audio/1-17808-A-12.wav,False,ESC-50


In [6]:
true_count=0
false_count=0
for i in TotData['Cough'] :
    if i == True :
        true_count+=1
    if i == False :
        false_count+=1
print(true_count,false_count)

18414 13474


In [7]:
TotData = TotData.sample(frac=1).reset_index(drop=True)

In [8]:
sup_Data = TotData.iloc[:1000, :]

In [9]:
con_Data = TotData.iloc[1000:, :]

In [10]:
true_count=0
false_count=0
for i in sup_Data['Cough'] :
    if i == True :
        true_count+=1
    if i == False :   
        false_count+=1
print(true_count,false_count)

577 423


In [11]:
sup_custpath  = './dataset/sup_custom_dataset_1000/'
def sup_feature_extractor(row):
    name     = row[0]
    try:
        audio,sr = librosa.load(row[1])
        #Mel Spectogram
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        melspec  = librosa.feature.melspectrogram(y=audio,sr=sr)
        s_db     = librosa.power_to_db(melspec, ref=np.max)
        librosa.display.specshow(s_db)

        savepath = os.path.join(sup_custpath,name+'.png')
        pylab.savefig(savepath, bbox_inches=None, pad_inches=0)
        pylab.close()
    except:
        print('File cannot open')
        return None,None
    return savepath

sup_imgpaths = []

In [12]:
con_custpath  = './dataset/con_custom_dataset_1000/'
def feature_extractor(row):
    name     = row[0]
    try:
        
        
        #Mel Spectogram
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        melspec  = librosa.feature.melspectrogram(y=audio,sr=sr)
        s_db     = librosa.power_to_db(melspec, ref=np.max)
        librosa.display.specshow(s_db)

        savepath = os.path.join(con_custpath,name+'.png')
        pylab.savefig(savepath, bbox_inches=None, pad_inches=0)
        pylab.close()
    except:
        print('File cannot open')
        return None,None
    return savepath


con_imgpaths = []

In [13]:
sup_Data

,ID,DIR,Cough,DataSet
0,f30664a5-7725-4cd7-b72c-9b0f2644935c,./public_dataset/f30664a5-7725-4cd7-b72c-9b0f2...,True,coughvid
1,AaTqzQjwIYaQW0MHufWVP5GWQ4I3,../Coswara-Data-master/20200416/AaTqzQjwIYaQW0...,True,coswara
2,67090bb0-377e-492c-9c49-8a608f50345d,./public_dataset/67090bb0-377e-492c-9c49-8a608...,True,coughvid
3,DeMaRSg6lVO7oA6v9HK3mbWjr3r2,../Coswara-Data-master/20200502/DeMaRSg6lVO7oA...,False,coswara
4,Q82KnSDEDTc0w7CDj1fSBntRTyg1,../Coswara-Data-master/20200919/Q82KnSDEDTc0w7...,False,coswara
...,...,...,...,...
995,ZgS6Ls08eFaArY1QATUpB76t7312,../Coswara-Data-master/20210406/ZgS6Ls08eFaArY...,False,coswara
996,2c148a1a-099b-48dd-97c2-437ecb15d430,./public_dataset/2c148a1a-099b-48dd-97c2-437ec...,True,coughvid
997,253084,../ESC-50-master/audio/3-253084-C-2.wav,False,ESC-50
998,273259b5-7350-4d15-b867-c88b46f552cc,./public_dataset/273259b5-7350-4d15-b867-c88b4...,True,coughvid


In [14]:
for row in tqdm(sup_Data.values):
    savepath  = sup_feature_extractor(row)
    sup_imgpaths.append(savepath)

 25%|██▌       | 252/1000 [01:34<04:37,  2.69it/s]

File cannot open


 41%|████▏     | 414/1000 [02:36<03:16,  2.98it/s]

File cannot open


 48%|████▊     | 481/1000 [03:02<03:38,  2.37it/s]

...

 69%|██████▉   | 689/1000 [04:21<01:52,  2.76it/s]

File cannot open


 94%|█████████▍| 942/1000 [06:05<00:33,  1.72it/s]

File cannot open


100%|██████████| 1000/1000 [06:27<00:00,  2.58it/s]


In [17]:
sup_imgpaths

['./dataset/sup_custom_dataset_2000/ef0dfde5-ec56-4b9b-b8d3-12429a9431bb.png',
 './dataset/sup_custom_dataset_2000/182025.png',
 './dataset/sup_custom_dataset_2000/a91ce163-1e19-4802-b210-c5a6e3c7cbd0.png',
 './dataset/sup_custom_dataset_2000/fNZ3MnqiGRQlpOptmB2S09fDdYi1.png',
 './dataset/sup_custom_dataset_2000/981e8854-4893-4215-82c8-589142cb5ae8.png',
 './dataset/sup_custom_dataset_2000/hVZQcrz6cfaOMfL3jpRqJBUiumF2.png',
 './dataset/sup_custom_dataset_2000/dLWRBpUZo7ddqJup4t3C2TwiVqA3.png',
 './dataset/sup_custom_dataset_2000/7R6vvnCDQcVBjNyh6TGtlx48eAZ2.png',
 './dataset/sup_custom_dataset_2000/188003.png',
 './dataset/sup_custom_dataset_2000/6178ef4e-172a-4b4a-8cc6-00dde9467d75.png',
 './dataset/sup_custom_dataset_2000/913c1096-6986-4770-98e7-ea5dde277b44.png',
 './dataset/sup_custom_dataset_2000/f3abf042-e22d-4e27-af5b-6650cc8f4883.png',
 './dataset/sup_custom_dataset_2000/B4YY3S8iF0husgCd1Y0tKZyMpgD3.png',
 './dataset/sup_custom_dataset_2000/BAMibuHkXaNr3JNc0Cvbu3kdSIp1.png',
 '

In [15]:
isnone = lambda x: x != (None,None)
sup_label  = lambda xx: 1 if xx == True else 0
sup_cast_x = list(map(isnone,sup_imgpaths))
sup_data_y = list(map(sup_label,sup_Data['Cough']))
sup_data_x= [sup_imgpaths[i] for i in range(len(sup_imgpaths)) if sup_cast_x[i] == True]
sup_data_y = [sup_data_y[i] for i in range(len(sup_imgpaths)) if sup_cast_x[i] == True]

In [16]:
sup_indices = np.arange(len(sup_data_x))
NUM_shuf= 5
sup_DATA    = {i:{} for i in range(NUM_shuf)}

for i in range(NUM_shuf):
    np.random.shuffle(sup_indices)
    sup_DATA[i]['MEL']   = [sup_data_x[i] for i in sup_indices]
    sup_DATA[i]['LABELS']= np.array([sup_data_y[i] for i in sup_indices])

In [17]:
len(sup_DATA[0]['MEL'])

994

In [18]:
import pickle
pickle.dump(sup_DATA, open( "sup_loaded_data_1000.pickle", "wb" ) )

In [19]:
for row in tqdm(con_Data.values):
    savepath  = feature_extractor(row)
    con_imgpaths.append(savepath)

  0%|          | 77/30888 [00:29<3:36:53,  2.37it/s]

File cannot open


  1%|          | 218/30888 [01:27<3:38:10,  2.34it/s]

File cannot open


  1%|          | 331/30888 [02:19<5:04:26,  1.67it/s]

...

 98%|█████████▊| 30367/30888 [3:42:41<04:18,  2.01it/s]

File cannot open


 99%|█████████▊| 30464/30888 [3:43:59<03:22,  2.10it/s]  

File cannot open


100%|██████████| 30888/30888 [3:47:17<00:00,  2.26it/s]


In [36]:
isnone = lambda x: x != (None,None)
cast_x = list(map(isnone,con_imgpaths))
data_x= [con_imgpaths[i] for i in range(len(con_imgpaths)) if cast_x[i] == True]

In [37]:
indices = np.arange(len(data_x))
NUM_shuf= 5
con_DATA    = {i:{} for i in range(NUM_shuf)}

for i in range(NUM_shuf):
    np.random.shuffle(indices)
    con_DATA[i]['MEL']   = [data_x[i] for i in indices]

In [38]:
pickle.dump( con_DATA, open( "con_loaded_data_2000.pickle", "wb" ) )

In [39]:
con_DATA

{0: {'MEL': ['./dataset/con_custom_dataset_2000/tpCCClZhHxXqu31XbOhexT55KdP2.png',
   './dataset/con_custom_dataset_2000/FNjPKyX47sUuQlzMc1urJ3VB3YE3.png',
   './dataset/con_custom_dataset_2000/FYINlRe3mfOO34R8QcT3skESVRl1.png',
   './dataset/con_custom_dataset_2000/IQ7tfWcSsRXGe0J94ouY9rsTB8V2.png',
   './dataset/con_custom_dataset_2000/PWXapCML3JSGefaCRAxKjCNp0Ps2.png',
   './dataset/con_custom_dataset_2000/bc1966e7-08b9-4399-a2b9-77030ffa05fd.png',
   './dataset/con_custom_dataset_2000/1gKYFXop8UQtz5vrzrbVcnjX0Vi1.png',
   './dataset/con_custom_dataset_2000/e02b509a-d93a-400f-9755-479ce02a53db.png',
   './dataset/con_custom_dataset_2000/36a9d918-5d78-4371-854c-a7f296627f3f.png',
   './dataset/con_custom_dataset_2000/e5xiRpxobLe69psjXzZtitwdxpg1.png',
   './dataset/con_custom_dataset_2000/aBhVajwUynU2XvqHDXl7EA9bVNm1.png',
   './dataset/con_custom_dataset_2000/OL4W8fQRM8POiI9lasCM9tgHUvG2.png',
   './dataset/con_custom_dataset_2000/RNnxsLNUWPeqrfpPcDb4b903fpj2.png',
   './dataset/con